Multi-platform loading

In [1]:
###SETTINGS###

import os
import re
import time
import subprocess
import shutil
from getpass import getpass
from urllib.parse import urlencode

# Platform detection
ON_KAGGLE = os.path.exists('/kaggle')
ON_COLAB = 'COLAB_RELEASE_TAG' in os.environ or os.path.exists('/content')
ON_VAST = any(k in os.environ for k in ("VAST_CONTAINERLABEL", "VAST_TCP_PORT_22", "CONTAINER_ID")) or os.path.exists('/workspace')

if shutil.which("aria2c") is None:
    print("aria2c not found → installing...")
    try:
        subprocess.run(["apt", "update", "-qq"], check=True, capture_output=True)
        result = subprocess.run(["apt", "install", "-y", "-qq", "aria2"], capture_output=True, text=True)
        if result.returncode == 0:
            print("aria2c installed successfully")
        else:
            print("Install failed (code {}):".format(result.returncode))
            print("stderr:", result.stderr.strip())
    except subprocess.CalledProcessError as e:
        print(f"apt error (code {e.returncode}): {e.stderr}")
    except Exception as e:
        print(f"Unexpected error: {e}")
else:
    print("aria2c already available")

# Determining the working directory
possible_bases = [
    "/workspace",       # Vast.ai / RunPod
    "/kaggle/working",  # Kaggle
    "/content",         # Google Colab
]

BASE_DIR = None
for path in possible_bases:
    if os.path.isdir(path):
        BASE_DIR = path
        break

if BASE_DIR is None:
    BASE_DIR = os.getcwd()
    print("WARNING: Known directory not found:", BASE_DIR)

print("Working directory:", BASE_DIR)

# Configuration
FORGE_DIR        = os.path.join(BASE_DIR, "stable-diffusion-webui-forge")
MODELS_DIR       = os.path.join(BASE_DIR, "stable-diffusion-webui-forge", "models", "Stable-diffusion")
LORA_DIR         = os.path.join(BASE_DIR, "stable-diffusion-webui-forge", "models", "Lora")
CONTROLNET_DIR   = os.path.join(FORGE_DIR, "extensions", "sd-webui-controlnet")
CONTROLNET_MODELS_DIR = os.path.join(CONTROLNET_DIR, "models")
EXTENSIONS_DIR   = os.path.join(FORGE_DIR, "extensions")
OUTPUTS_DIR      = os.path.join(FORGE_DIR, "outputs")

for d in [MODELS_DIR, LORA_DIR, CONTROLNET_DIR, CONTROLNET_MODELS_DIR, EXTENSIONS_DIR, OUTPUTS_DIR]:
    os.makedirs(d, exist_ok=True)


def get_secret(name: str):
    """Get secret from env/Kaggle/Colab, then optional prompt."""
    value = os.environ.get(name)
    if value:
        return value.strip(), "env"

    # Kaggle secrets
    if ON_KAGGLE:
        try:
            from kaggle_secrets import UserSecretsClient
            value = UserSecretsClient().get_secret(name)
            if value:
                return value.strip(), "kaggle_secrets"
        except Exception:
            pass

    # Colab secrets panel: from google.colab import userdata
    if ON_COLAB:
        try:
            from google.colab import userdata
            value = userdata.get(name)
            if value:
                return value.strip(), "colab_userdata"
        except Exception:
            pass

    # Manual fallback for Vast.ai/local
    try:
        manual = getpass(f"{name}: ").strip() or None
        if manual:
            return manual, "manual_input"
    except Exception:
        pass

    return None, None


CIVITAI_TOKEN, CIVITAI_SRC = get_secret("CIVITAI_TOKEN")
HF_TOKEN, HF_SRC = get_secret("HF_TOKEN")

TOKENS = {}
if CIVITAI_TOKEN:
    TOKENS["CIVITAI"] = CIVITAI_TOKEN
if HF_TOKEN:
    TOKENS["HF_TOKEN"] = HF_TOKEN

print("Token sources:")
print(f"  CIVITAI_TOKEN: {CIVITAI_SRC or 'not found'}")
print(f"  HF_TOKEN: {HF_SRC or 'not found'}")
if ON_VAST:
    print("Vast.ai tip: add CIVITAI_TOKEN/HF_TOKEN in template env vars, restart container, then rerun this cell.")

if not CIVITAI_TOKEN:
    print("CivitAI token not found")
if not HF_TOKEN:
    print("HF token not found")


def _prepare_download_url(url, token):
    """CivitAI download works more reliably with token as query param."""
    if token and "civitai.com/api/download/models" in url and "token=" not in url:
        sep = "&" if "?" in url else "?"
        return f"{url}{sep}{urlencode({'token': token})}"
    return url


def _looks_valid_file(path, min_bytes=1_000_000):
    return os.path.exists(path) and os.path.getsize(path) > min_bytes


def _human_mb(num_bytes):
    return f"{num_bytes / (1024 * 1024):.1f} MB"


def _estimate_expected_mb(label):
    # Examples: "151 MB", "6,46 GB"
    match = re.search(r"(\d+[\.,]?\d*)\s*(MB|GB)", label, re.IGNORECASE)
    if not match:
        return None
    value = float(match.group(1).replace(',', '.'))
    unit = match.group(2).upper()
    return value * (1024 if unit == "GB" else 1)


def _size_sanity_warning(path, expected_mb, tolerance=0.7):
    if expected_mb is None or not os.path.exists(path):
        return
    actual_mb = os.path.getsize(path) / (1024 * 1024)
    if actual_mb < expected_mb * tolerance:
        print(f"  WARNING: file size looks low ({actual_mb:.1f} MB vs expected ~{expected_mb:.1f} MB)")


def download_file(url, output_path, token=None, expected_mb=None):
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    if _looks_valid_file(output_path):
        print(f"  Already ready → {_human_mb(os.path.getsize(output_path))}")
        return

    token = token.strip() if isinstance(token, str) else token
    final_url = _prepare_download_url(url, token)

    attempts = []

    if shutil.which("aria2c"):
        aria2_cmd = [
            "aria2c", "-x", "16", "-s", "16", "-k", "1M",
            "--summary-interval=1", "--console-log-level=warn",
            "--max-tries=3", "--retry-wait=2", "--continue=true",
            "-d", os.path.dirname(output_path),
            "-o", os.path.basename(output_path),
            final_url,
        ]
        if token:
            aria2_cmd += ["--header", f"Authorization: Bearer {token}"]
        attempts.append(("aria2c", aria2_cmd))

    if shutil.which("curl"):
        curl_cmd = [
            "curl", "-L", "-C", "-", "--fail", "--progress-bar",
            "--retry", "3", "--retry-delay", "2",
            "-A", "Mozilla/5.0",
            final_url, "-o", output_path,
        ]
        if token:
            curl_cmd = curl_cmd[:1] + ["-H", f"Authorization: Bearer {token}"] + curl_cmd[1:]
        attempts.append(("curl", curl_cmd))

    wget_cmd = [
        "wget", "-c", final_url, "-O", output_path,
        "--tries=2", "--waitretry=2",
        "--user-agent=Mozilla/5.0",
        "--progress=dot:giga",
    ]
    if token:
        wget_cmd = wget_cmd[:2] + ["--header", f"Authorization: Bearer {token}"] + wget_cmd[2:]
    attempts.append(("wget", wget_cmd))

    last_error = None
    for tool_name, cmd in attempts:
        try:
            start = time.time()
            print(f"  [{tool_name}] started")
            subprocess.run(cmd, check=True)
            elapsed = max(time.time() - start, 0.01)

            if not _looks_valid_file(output_path):
                raise RuntimeError(f"{tool_name} finished but file is empty/small")

            size_bytes = os.path.getsize(output_path)
            speed_mbps = (size_bytes * 8) / elapsed / 1_000_000
            print(f"  OK → {os.path.basename(output_path)} ({tool_name}, {_human_mb(size_bytes)}, ~{speed_mbps:.1f} Mbit/s)")
            _size_sanity_warning(output_path, expected_mb)
            return
        except Exception as e:
            last_error = e
            if os.path.exists(output_path) and os.path.getsize(output_path) < 1_000_000:
                try:
                    os.remove(output_path)
                except Exception:
                    pass
            print(f"  {tool_name} failed: {e}")

    raise RuntimeError(f"All download methods failed for {output_path}. Last error: {last_error}")


aria2c not found → installing...
aria2c installed successfully
Working directory: /kaggle/working


In [2]:
### CONTROLNET INSTALL OPTIONS ###

# Option A (default): force clean reinstall
# Option B: set CONTROLNET_INSTALL_MODE = "update" for git pull in existing repo
# Option C: set CONTROLNET_INSTALL_MODE = "skip" to keep current state

CONTROLNET_REPO_URL = "https://github.com/Mikubill/sd-webui-controlnet"
CONTROLNET_INSTALL_MODE = os.environ.get("CONTROLNET_INSTALL_MODE", "reinstall").strip().lower()

if CONTROLNET_INSTALL_MODE not in {"reinstall", "update", "skip"}:
    raise ValueError("CONTROLNET_INSTALL_MODE must be one of: reinstall, update, skip")

print(f"ControlNet extension path: {CONTROLNET_DIR}")
print(f"Install mode: {CONTROLNET_INSTALL_MODE}")

if CONTROLNET_INSTALL_MODE == "skip":
    print("ControlNet install skipped")
else:
    if os.path.isdir(CONTROLNET_DIR) and CONTROLNET_INSTALL_MODE == "reinstall":
        print("Removing existing ControlNet directory...")
        shutil.rmtree(CONTROLNET_DIR)

    if not os.path.isdir(CONTROLNET_DIR):
        print("Cloning ControlNet repository...")
        subprocess.run(["git", "clone", CONTROLNET_REPO_URL, CONTROLNET_DIR], check=True)
    else:
        print("Updating ControlNet repository...")
        subprocess.run(["git", "-C", CONTROLNET_DIR, "pull", "--ff-only"], check=True)

os.makedirs(CONTROLNET_MODELS_DIR, exist_ok=True)
print("ControlNet repository is ready")
print(f"ControlNet models directory: {CONTROLNET_MODELS_DIR}")

ControlNet extension path: /kaggle/working/stable-diffusion-webui-forge/extensions/sd-webui-controlnet
Install mode: reinstall
Removing existing ControlNet directory...
Cloning ControlNet repository...


Cloning into '/kaggle/working/stable-diffusion-webui-forge/extensions/sd-webui-controlnet'...


ControlNet repository is ready
ControlNet models directory: /kaggle/working/stable-diffusion-webui-forge/extensions/sd-webui-controlnet/models


In [3]:

#ControlNET models download

controlnet_models_to_download = [
    ("t2i-adapter_xl_openpose 151 MB", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_xl_openpose.safetensors", "t2i-adapter_xl_openpose.safetensors", "HF_TOKEN"),
    ("t2i-adapter_xl_canny 148 MB", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_xl_canny.safetensors", "t2i-adapter_xl_canny.safetensors", "HF_TOKEN"),
    ("t2i-adapter_xl_sketch 148 MB", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_xl_sketch.safetensors", "t2i-adapter_xl_sketch.safetensors", "HF_TOKEN"),
    ("t2i-adapter_diffusers_xl_depth_midas 151 MB", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_diffusers_xl_depth_midas.safetensors", "t2i-adapter_diffusers_xl_depth_midas.safetensors", "HF_TOKEN"),
    ("t2i-adapter_diffusers_xl_depth_zoe 151 MB", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_diffusers_xl_depth_zoe.safetensors", "t2i-adapter_diffusers_xl_depth_zoe.safetensors", "HF_TOKEN"),
    ("t2i-adapter_diffusers_xl_lineart 151 MB", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_diffusers_xl_lineart.safetensors", "t2i-adapter_diffusers_xl_lineart.safetensors", "HF_TOKEN"),
    #("", "", "", ""),
]

print(f"Total Models/checkpoints in the list: {len(controlnet_models_to_download)}")
skipped = 0

print(f"All Models/checkpoints: {len(controlnet_models_to_download)}")
skipped = 0

for name, url, filename, source in controlnet_models_to_download:
    output_path = os.path.join(CONTROLNET_MODELS_DIR, filename)
    
    if os.path.exists(output_path) and os.path.getsize(output_path) > 1_000_000:
        print(f"  Already has → {name:<30} {filename}")
        skipped += 1
        continue
    
    token = TOKENS.get(source)
    
    print(f"  Loading {name:<30} ({source}) → {filename}")
    expected_mb = _estimate_expected_mb(name)
    try:
        download_file(url=url, output_path=output_path, token=token, expected_mb=expected_mb)
    except Exception as e:
        print(f"  ERROR: {e}")

print(f"\n DONE. New: {len(controlnet_models_to_download) - skipped}, skipped: {skipped}")


Total Models/checkpoints in the list: 6
All Models/checkpoints: 6
  Loading t2i-adapter_xl_openpose 151 MB (HF_TOKEN) → t2i-adapter_xl_openpose.safetensors
  OK → t2i-adapter_xl_openpose.safetensors (aria2c)
  Loading t2i-adapter_xl_canny 148 MB    (HF_TOKEN) → t2i-adapter_xl_canny.safetensors
  OK → t2i-adapter_xl_canny.safetensors (aria2c)
  Loading t2i-adapter_xl_sketch 148 MB   (HF_TOKEN) → t2i-adapter_xl_sketch.safetensors
  OK → t2i-adapter_xl_sketch.safetensors (aria2c)
  Loading t2i-adapter_diffusers_xl_depth_midas 151 MB (HF_TOKEN) → t2i-adapter_diffusers_xl_depth_midas.safetensors
  OK → t2i-adapter_diffusers_xl_depth_midas.safetensors (aria2c)
  Loading t2i-adapter_diffusers_xl_depth_zoe 151 MB (HF_TOKEN) → t2i-adapter_diffusers_xl_depth_zoe.safetensors
  OK → t2i-adapter_diffusers_xl_depth_zoe.safetensors (aria2c)
  Loading t2i-adapter_diffusers_xl_lineart 151 MB (HF_TOKEN) → t2i-adapter_diffusers_xl_lineart.safetensors
  OK → t2i-adapter_diffusers_xl_lineart.safetensors (a

In [4]:

#controlnet models to download

models_to_download = [
    ("WAI ILL V16.0 6,46 GB", "https://civitai.com/api/download/models/2514310?type=Model&format=SafeTensor&size=pruned&fp=fp16", "waiIllustriousSDXL_v160.safetensors", "CIVITAI"),
]

print(f"Total Models/checkpoints in the list: {len(models_to_download)}")
skipped = 0

print(f"All Models/checkpoints: {len(models_to_download)}")
skipped = 0

for name, url, filename, source in models_to_download:
    output_path = os.path.join(MODELS_DIR, filename)
    
    if os.path.exists(output_path) and os.path.getsize(output_path) > 1_000_000:
        print(f"  Already has → {name:<30} {filename}")
        skipped += 1
        continue
    
    token = TOKENS.get(source)
    
    print(f"  Loading {name:<30} ({source}) → {filename}")
    expected_mb = _estimate_expected_mb(name)
    try:
        download_file(url=url, output_path=output_path, token=token, expected_mb=expected_mb)
    except Exception as e:
        print(f"  ERROR: {e}")

print(f"\n DONE. New: {len(models_to_download) - skipped}, skipped: {skipped}")


Total Models/checkpoints in the list: 1
All Models/checkpoints: 1
  Loading WAI ILL V16.0 6,46 GB          (CIVITAI) → waiIllustriousSDXL_v160.safetensors
  OK → waiIllustriousSDXL_v160.safetensors (curl)

 DONE. New: 1, skipped: 0


In [5]:

#LoRa

lora_to_download = [
    ("Detailer IL V2 218 MB",        "https://civitai.com/api/download/models/1736373?type=Model&format=SafeTensor",    "detailer_v2_il.safetensors",     "CIVITAI"),
    ("Realistic filter V1 55 MB",    "https://civitai.com/api/download/models/1124771?type=Model&format=SafeTensor",    "realistic_filter_v1_il.safetensors", "CIVITAI"),
    ("Hyperrealistic V4 ILL 435 MB", "https://civitai.com/api/download/models/1914557?type=Model&format=SafeTensor",    "hyperrealistic_v4_ill.safetensors",  "CIVITAI"),
    ("Niji semi realism V3.5 ILL 435 MB", "https://civitai.com/api/download/models/1882710?type=Model&format=SafeTensor", "niji_semi_realism_v35.safetensors", "CIVITAI"), 
    ("ATNR Style ILL V1.1 350 MB", "https://civitai.com/api/download/models/1711464?type=Model&format=SafeTensor", "atnr_style_ill_v1.1.safetensors", "CIVITAI"), 
    ("Face Enhancer Ill 218 MB", "https://civitai.com/api/download/models/1839268?type=Model&format=SafeTensor", "face_enhancer_ill.safetensors", "CIVITAI"),
    ("Smooth Detailer Booster V4 243 MB", "https://civitai.com/api/download/models/2196453?type=Model&format=SafeTensor", "smooth_detailer_booster_v4.safetensors", "CIVITAI"),
    ("USNR Style V-pred 157 MB", "https://civitai.com/api/download/models/2555444?type=Model&format=SafeTensor", "usnr_style_v_pred.safetensors", "CIVITAI"),
    ("748cm Style V1 243 MB", "https://civitai.com/api/download/models/1056404?type=Model&format=SafeTensor", "748cm_style_v1.safetensors", "CIVITAI"),
    ("Velvet's Mythic Fantasy Styles IL 218 MB", "https://civitai.com/api/download/models/2620790?type=Model&format=SafeTensor", "velvets_mythic_fantasy_styles_il.safetensors", "CIVITAI"),
    #("", "", "", "CIVITAI"),
]

print(f"Total LoRa in the list: {len(lora_to_download)}")
skipped = 0

print(f"All LoRA: {len(lora_to_download)}")
skipped = 0

for name, url, filename, source in lora_to_download:
    output_path = os.path.join(LORA_DIR, filename)
    
    if os.path.exists(output_path) and os.path.getsize(output_path) > 1_000_000:
        print(f"  Already has → {name:<30} {filename}")
        skipped += 1
        continue
    
    token = TOKENS.get(source)
    
    print(f"  Loading {name:<30} ({source}) → {filename}")
    expected_mb = _estimate_expected_mb(name)
    try:
        download_file(url=url, output_path=output_path, token=token, expected_mb=expected_mb)
    except Exception as e:
        print(f"  ERROR: {e}")

print(f"\n DONE. New: {len(lora_to_download) - skipped}, skipped: {skipped}")


Total LoRa in the list: 10
All LoRA: 10
  Loading Detailer IL V2 218 MB          (CIVITAI) → detailer_v2_il.safetensors
  OK → detailer_v2_il.safetensors (curl)
  Loading Realistic filter V1 55 MB      (CIVITAI) → realistic_filter_v1_il.safetensors
  OK → realistic_filter_v1_il.safetensors (curl)
  Loading Hyperrealistic V4 ILL 435 MB   (CIVITAI) → hyperrealistic_v4_ill.safetensors
  OK → hyperrealistic_v4_ill.safetensors (curl)
  Loading Niji semi realism V3.5 ILL 435 MB (CIVITAI) → niji_semi_realism_v35.safetensors
  OK → niji_semi_realism_v35.safetensors (curl)
  Loading ATNR Style ILL V1.1 350 MB     (CIVITAI) → atnr_style_ill_v1.1.safetensors
  OK → atnr_style_ill_v1.1.safetensors (curl)
  Loading Face Enhancer Ill 218 MB       (CIVITAI) → face_enhancer_ill.safetensors
  OK → face_enhancer_ill.safetensors (curl)
  Loading Smooth Detailer Booster V4 243 MB (CIVITAI) → smooth_detailer_booster_v4.safetensors
  OK → smooth_detailer_booster_v4.safetensors (curl)
  Loading USNR Style V-

In [ ]:
# Archive outputs for download (folders like DD_MM_YYYY with PNG files)

import os
import shutil
from datetime import datetime

archive_source = OUTPUTS_DIR
stamp = datetime.now().strftime("%d_%m_%Y_%H%M")
archive_base = os.path.join(BASE_DIR, f"outputs_backup_{stamp}")
archive_zip = shutil.make_archive(archive_base, "zip", archive_source)

size_mb = os.path.getsize(archive_zip) / (1024 * 1024)
print(f"Archive created: {archive_zip}")
print(f"Archive size: {size_mb:.1f} MB")

if ON_COLAB:
    from google.colab import files
    files.download(archive_zip)
    print("Download started via Colab files.download")
elif ON_KAGGLE:
    print("Kaggle: open the Files panel and download this .zip manually.")
else:
    print("Vast.ai/local: download the .zip from your workspace or Jupyter file browser.")
